In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
training_set=pd.read_csv('../input/application_train.csv')

In [ ]:
test_set=pd.read_csv('../input/application_test.csv')

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
le=LabelEncoder()
le_count=0
for col in training_set:
    if training_set[col].dtype=='object':
        if len(list(training_set[col].unique()))<=2:
            le.fit(training_set[col])
            print(f'this {col} has {le.classes_}')
            training_set[col]=le.transform(training_set[col])
            test_set[col]=le.transform(test_set[col])
            le_count+=1
print(f'{le_count} columns were label encoded')

In [ ]:
training_set=pd.get_dummies(training_set)
test_set=pd.get_dummies(test_set)

In [ ]:
training_labels=training_set['TARGET']

In [ ]:
training_set, test_set=training_set.align(test_set, join='inner', axis=1)
print(f'training features shape: {training_set.shape}')
print(f'test features shape: {test_set.shape}')

In [ ]:
training_set['TARGET']=training_labels

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

In [ ]:
X_train=training_set.drop(columns=['TARGET'])
X_test=test_set.copy()
features=list(X_train.columns)
imputer=SimpleImputer(strategy='median')
scaler=MinMaxScaler(feature_range=(0,1))
imputer.fit(X_train)
X_train=imputer.transform(X_train)
X_test=imputer.transform(X_test)
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)
print(f'training data shape:{X_train.shape}')
print(f'test data shape:{X_test.shape}')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier=RandomForestClassifier(n_estimators=50)
classifier.fit(X_train, training_labels)

In [ ]:
classifier_pred=classifier.predict_proba(X_test)[:,1]

In [ ]:
def show_feature_importances(model, features):
    plt.figure(figsize=(12,8))
    results=pd.DataFrame({'feature': features, 'importance': model.feature_importances_})
    results=results.sort_values(by='importance', ascending=False)
    print(results.head(10))
    print('\nNuber of features with importance greater than 0.01= ', np.sum(results['importance']>0.01))
    results.head(20).plot(x='feature', y='importance', kind='barh', color='red', edgecolor='k', title='Feature Importances')
    return results

In [ ]:
feature_importances=show_feature_importances(classifier, features)

In [ ]:
submit=test_set[['SK_ID_CURR']]
submit['TARGET']=classifier_pred

In [ ]:
submit.to_csv('random_forest_baseline_v1.csv', index=False)